# **1.Data loading**

## Imports

In [1]:
from os import path, sep, pardir, getcwd, listdir
from typing import AnyStr, List, Dict, Set, Any

from pandas import DataFrame, read_csv, concat

## Data loading class

*(Completetly unnecessary and a bite chaotic)*

In [2]:
from os import path, sep, pardir, getcwd, listdir
from typing import AnyStr, List, Dict, Set, Any

from pandas import DataFrame, read_csv, concat


class LoadedData:
    """Class storing the train and test data"""

    @staticmethod
    def _get_loading_paths(searched_tags: Set[AnyStr] = ('train', 'test', 'train_updates')) -> Dict[AnyStr, AnyStr]:
        """Function returns the loading paths for the relevant data"""


        def _join_list_to_str(*args: List[Any]) -> str:
            """Function joins the first list arguments"""
            str_var = ""

            for arg in args:
                if arg != args[-1]:
                    str_var += arg[0] + '\\'

                else:
                    str_var += arg[0]

            return str_var


        def _check_list(list_var: List[AnyStr]) -> bool:
            """Function checks the given list length"""
            return True if len(list_var) == 1 else False


        def _get_working_directory(str_var: AnyStr = "") -> AnyStr:
            """Function returns the path to data in the working directory"""
            return str(path.normpath(getcwd() + sep + pardir)) + f'\\{str_var}'


        def _get_all_subdirectories(path_str: AnyStr = "") -> List[AnyStr]:
            """Function returns all the possible subdirectories"""
            if path_str:
                return listdir(path_str)

            else:
                return listdir(_get_working_directory())

        docs_subdir_list = [subdir for subdir in _get_all_subdirectories() if subdir.lower().find('docs') >= 0]

        if _check_list(docs_subdir_list):
            docs_path = _get_working_directory(_join_list_to_str(docs_subdir_list))
            data_subdir_list = [subdir for subdir in _get_all_subdirectories(docs_path)
                                if subdir.lower().find('data') >= 0]

            if _check_list(data_subdir_list):
                data_path = _get_working_directory(_join_list_to_str(docs_subdir_list, data_subdir_list))
                tag_dict = {}

                for tag in searched_tags:
                    tag_subdir_list = [subdir for subdir in _get_all_subdirectories(data_path)
                                       if subdir.lower().find(f'{tag}') >= 0]

                    if len(tag_subdir_list) > 0:
                        tag_path = _get_working_directory(_join_list_to_str(docs_subdir_list,
                                                                            data_subdir_list,
                                                                            tag_subdir_list))
                        tag_dict[f'{tag}_df'] = tag_path

                return tag_dict

            else:
                raise FileExistsError('No or to much of data directory in the docs')

        else:
            raise FileExistsError('No or to much of docs directory in the current working directory')

    @staticmethod
    def _get_corrected_train_data(train_df: DataFrame, update_df: DataFrame) -> DataFrame:
        """Function returns the corrected train data with the last update file"""
        update_df.data_source = train_df.loc[update_df.index, 'data_source']
        update_indexes = update_df[update_df.protein_sequence.isna()].index
        update_df.loc[update_indexes, 'protein_sequence'] = train_df.loc[update_indexes, 'protein_sequence']
        train_df.drop(update_df.index, inplace=True)
        return concat([train_df, update_df])


    def __init__(self, initialzation: bool = True) -> None:
        if initialzation is True:
            self.loading_paths_dict = self._get_loading_paths()
            self.test_df = read_csv(self.loading_paths_dict['test_df'], index_col="seq_id")
            self.train_df = self._get_corrected_train_data(
                read_csv(self.loading_paths_dict['train_df'], index_col="seq_id"),
                read_csv(self.loading_paths_dict['train_updates_df'], index_col="seq_id")
            )

## Class calling

In [3]:
Loaded_data = LoadedData()

## Verification of loading

### Train dataframe

In [4]:
Loaded_data.train_df.head()

,protein_sequence,pH,data_source,tm
seq_id,,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


### Test dataframe

In [5]:
Loaded_data.test_df.head()

,protein_sequence,pH,data_source
seq_id,,,
31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes
31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes


# **Data cleaning**

## Data cleaning class

In [16]:
class CleanedData(LoadedData):
    """Class for cleaning and storing the cleaned data"""

    @staticmethod
    def _remove_empty_rows(df: DataFrame, col_name: str) -> None:
        df.drop(df[df[col_name].isna()].index, inplace=True)


    def __init__(self, target_column: str = 'tm') -> None:
        super(CleanedData, self).__init__()
        # Removal of the irrelevant data
        for col_name in self.test_df.columns.to_list():
            if self.train_df[col_name].isin(self.test_df[col_name].unique()).sum() == 0\
                    and col_name != 'protein_sequence':
                if 'cleaned_train_data' not in locals():
                    self.cleaned_train_data = self.train_df.drop(col_name, axis=1, )
                    self.cleaned_test_data = self.test_df.drop(col_name, axis=1)

                else:
                    self.cleaned_train_data.drop(col_name, axis=1, inplace=True)
                    self.cleaned_test_data.drop(col_name, axis=1, inplace=True)

        self._remove_empty_rows(self.cleaned_train_data, target_column)

        del self.test_df, self.train_df

## Class initialization

In [8]:
Cleaned_data = CleanedData()

## Verification of loading

### Train data

In [12]:
Cleaned_data.cleaned_train_data.head()

,protein_sequence,pH,tm
seq_id,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5


In [13]:
Cleaned_data.cleaned_train_data.shape

(28981, 3)

### Test data

In [14]:
Cleaned_data.cleaned_test_data.head()

,protein_sequence,pH
seq_id,,
31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8
31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8
31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8
31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8
31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8


In [15]:
Cleaned_data.cleaned_test_data.shape

(2413, 2)